In [1]:
from typing import Union, Tuple, List
from transformers import BertConfig, BertForPreTraining, BertTokenizerFast
import numpy as np
import random
import pandas as pd
from typing import Union, Tuple, List
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
import numpy as np
import random
import pandas as pd
from datetime import datetime, date
from tqdm.notebook import tqdm
from sklearn.metrics import log_loss, accuracy_score, precision_score, recall_score
from tqdm import tqdm
# from IPython.display import Image
from joblib import Parallel, delayed
import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch
from transformers import BertTokenizer, BertModel
import re
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)
from torch.utils.data.dataset import Dataset
from transformers import DataCollatorForLanguageModeling
import os
from transformers import Trainer, TrainingArguments, AutoTokenizer
from transformers.utils import logging
from transformers import BertConfig, BertForPreTraining, BertTokenizerFast
from filelock import FileLock
import time
import pickle
from transformers import BertConfig, BertForPreTraining
from tokenizers import BertWordPieceTokenizer
import argparse

from transformers import Trainer, TrainingArguments, AutoTokenizer
from datetime import datetime, date
from tqdm.notebook import tqdm
from tqdm import tqdm
# from IPython.display import Image
from joblib import Parallel, delayed
import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch
from transformers import BertTokenizer, BertModel
import re
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)
from torch.utils.data.dataset import Dataset
import matplotlib.pyplot as plt
from transformers import DataCollatorForLanguageModeling
import os
from transformers import Trainer, TrainingArguments
from transformers.utils import logging
logger = logging.get_logger(__name__)
from filelock import FileLock
import time
import pickle

/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
wine_df = pd.read_csv('/opt/ml/wine/data/wine_df.csv')
review_df = pd.read_csv('/opt/ml/wine/data/review_df_total.csv',encoding = 'utf-8-sig').loc[:,['user_url','rating','text','wine_url']]

In [3]:
import json
with open('/opt/ml/wine/code/data/feature_map/item2idx.json','r') as f:
    item2idx = json.load(f)

In [ ]:
review_df = review_df[review_df['text'].isna()==False]

In [ ]:
review_df['text'] = review_df['text'].apply(lambda x: x + '.' if x[-1] != '.' else x)

In [ ]:
def keep_english_and_digits(text):
    # Remove any characters that are not English alphabets, digits, periods, or commas at the end of sentences
    clean_text = re.sub(r'[^a-zA-Z0-9\s.,]', '', text)
    return clean_text

In [ ]:
review_df['text'] = review_df['text'].apply(keep_english_and_digits)

In [ ]:
review_df['wine_id'] = review_df['wine_url'].map(item2idx)
review_df = review_df[review_df['wine_id'].isna()==False]
review_df['wine_id'] = review_df['wine_id'].astype('int').astype('category')

In [ ]:
note_df = wine_df.filter(like='_child')

In [ ]:
notes = {}
import ast
def str2dict(x):
    try: return ast.literal_eval(x)
    except: return {}
def get_keys(x):
    return set(x.keys())

for col in note_df.columns:
    note_df.loc[:,col] = note_df.loc[:,col].apply(str2dict)
    sub_note = set()
    for i in tqdm(range(len(note_df))):
        subs = get_keys(note_df[col][i])
        sub_note = sub_note | subs
    notes[col.replace('_child','')] = sub_note

In [ ]:
text_with_note = review_df.drop(['rating','wine_id'], axis = 1)

# 와인 기본 정보 multi label

In [ ]:
wine_basic_info = wine_df.loc[:, ['url','country','region','grape','winetype']]
wine_basic_info['wine_id'] = wine_basic_info['url'].map(item2idx)
wine_basic_info= wine_basic_info[wine_basic_info['wine_id'].isna()==False]
wine_basic_info['wine_id'] = wine_basic_info['wine_id'].astype('int').astype('category')

In [102]:
def check_note_in_review(text, notes_data):
    text = text.lower()
    result = []
    for key in notes:
        if any(word in text for word in notes[key]):
            result.append(1)
        else: result.append(0)
    return result

def marking_data(df, notes_data):
    df.reset_index(inplace = True)
    data = []
    for i in tqdm(range(len(df))):
        note_onehot = check_note_in_review(df.loc[i,'text'], notes_data)
        tmp = {}
        tmp['wine_id'] = df.loc[i,'wine_id']
        tmp['text'] = df.loc[i,'text']
        tmp['note_label'] = note_onehot
        data.append(tmp)
        
    return pd.DataFrame(data)


def parallel_dataframe_2input(func, df, notes_data, num_cpu):

    for key in notes_data: notes_data[key] = set(notes_data[key])
    chunks = np.array_split(df, num_cpu)

    print('Parallelizing with ' +str(num_cpu)+'cores')
    with Parallel(n_jobs = num_cpu, backend="multiprocessing") as parallel:
        results = parallel(delayed(func)(chunks[i], notes_data) for i in range(num_cpu))

    for i,data in enumerate(results):
        if i == 0:
            output = data
        else:
            output = pd.concat([output, data], axis=0)
    output.reset_index(inplace = True, drop = True)
    return output

In [103]:
with open('/opt/ml/wine/data/note.json','r') as f: notes_data = json.load(f)

In [105]:
text_with_notelabel = parallel_dataframe_2input(marking_data, review_df, notes_data, 8)

Parallelizing with 8cores


OSError: [Errno 12] Cannot allocate memory

In [138]:
text_with_notelabel.to_csv('/opt/ml/wine/data/text_with_notelabel.csv', index = False, encoding = 'utf-8')

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            truncation = True,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
print('학습 문장 최대 길이 :',max(len(l) for l in text_with_notelabel['text']))
print('학습 문장의 평균 길이 :',sum(map(len, text_with_notelabel['text']))/len(text_with_notelabel['text']))

plt.hist([len(s) for s in text_with_notelabel['text']], bins=50)
plt.xlabel('length of data')
plt.ylabel('number of data')
plt.show()

In [ ]:
# Creating the dataset and dataloader for the neural network
train_size = 0.8
train_dataset = text_with_notelabel.sample(frac=train_size,random_state=200)
test_dataset = text_with_notelabel.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


In [ ]:

print("FULL Dataset: {}".format(text_with_notelabel.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

training_set = CustomDataset(train_dataset, tokenizer, 156)
testing_set = CustomDataset(test_dataset, tokenizer, 156)

In [129]:
TRAIN_BATCH_SIZE = 512
VALID_BATCH_SIZE = 1024
EPOCHS = 50
LEARNING_RATE = 1e-05

In [130]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [131]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self, num_labels):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, num_labels)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
model = BERTClass(len(text_with_notelabel['label'][0]))
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [132]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [133]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [136]:
def train(epoch):
    model.train()
    for data in tqdm(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

In [45]:
basic_info = pd.read_csv('/opt/ml/wine/data/basic_info_total.csv')
wine_df = pd.read_csv('/opt/ml/wine/data/wine_df.csv')
wine_info = wine_df.loc[:, ['url','winetype']].merge(basic_info, on='url')

In [46]:
import ast
from collections import defaultdict

def str2list(x):
    try: return ast.literal_eval(x)
    except: return ['None']

def get_all_grapes(wine_info):
    
    unique_grape = defaultdict(int)
    for grapes in tqdm(wine_info['grapes']):
        for grape in grapes:
            grape = re.sub(r'\d+%_', '', grape).lower()
            unique_grape[grape] += 1

    return dict(sorted(unique_grape.items(), key=lambda x: x[1], reverse=True))

In [47]:
def grape_encoding(wine_info, grape2idx, threshold):
    wine_info.reset_index(drop = True, inplace= True)
    grape_onehot = []
    for grapes in wine_info['grapes']:
        tmp = [0 for _ in range(len(grape2idx))]
        for grape in grapes:
            if grape in grape2idx:
                matches = re.findall(r'(\d+)%_(\w+)', grape)

                if len(matches) == 1:
                    percentage, grape  = matches[0]
                    if int(percentage) >= threshold: tmp[grape2idx[grape]] = 1

                else: tmp[grape2idx[grape]] = 1
        grape_onehot.append(tmp)
    wine_info['grape_label'] = grape_onehot
    return wine_info

In [48]:
def cut_lowcount_feat(df, col, threshold):
    feats = {}
    idx = 1
    for feature, count in zip(df[col].value_counts().index,  df[col].value_counts()):
        if count >= threshold:
            feats[feature] = idx
            idx += 1
        else:
            feats[feature] = 0
    return feats

def string2label(df, column, feature2idx):
    labels = []
    df[column] = df[column].fillna("None")
    for feature in tqdm(df[column]):
        tmp = [0 for _ in range(max(feature2idx.values())+1)]

        if feature in feature2idx:
            tmp[feature2idx[feature]] = 1
        else: tmp[-1] = 0

        labels.append(tmp)
        
    df[f"{column}_label"] = labels
    return df


In [49]:
import json
with open('/opt/ml/wine/code/data/feature_map/item2idx.json','r') as f:
    item2idx = json.load(f)
wine_info['wine_id'] = wine_info['url'].map(item2idx)
wine_info = wine_info[wine_info['wine_id'].isna()==False]
wine_info['wine_id'] = wine_info['wine_id'].astype('int').astype('category')
wine_info = wine_info.loc[:,['wine_id','country','grapes','winetype']]

In [44]:
def gen_labeled_data(
        wine_info : pd.DataFrame, 
        grape_threshold : int = 89, 
        grape_percentage_threshold : int = 20,
        country_threshold : int = 150,
        winetype_threshold : int = 0
        ):
    wine_info.drop_duplicates(inplace = True)
    wine_info = wine_info.loc[:,['wine_id','country','grapes','winetype']]
    wine_info['country'].fillna('other', inplace = True)
    wine_info['winetype'].fillna('other', inplace = True)
    wine_info['grapes'] = wine_info['grapes'].apply(str2list)

    unique_grape = get_all_grapes(wine_info)
    filtered_grape = {k: v for k, v in unique_grape.items() if v >=grape_threshold}
    grape2idx =  {k: v for v, k in enumerate(filtered_grape.keys())}

    wine_info = grape_encoding(wine_info, grape2idx, grape_percentage_threshold)
    sum_zero_filter = lambda x: sum(x) != 0

    # Apply the filter and drop rows where the sum of the list is 0
    wine_info = wine_info[wine_info['grape_label'].apply(sum_zero_filter)]

    country2idx =  cut_lowcount_feat(wine_info, 'country', country_threshold)
    winetype2idx = cut_lowcount_feat(wine_info, 'winetype', winetype_threshold)

    wine_info = string2label(wine_info, 'winetype', winetype2idx)
    wine_info = string2label(wine_info, 'country', country2idx)
    wine_info.reset_index(inplace = True, drop = True)
    return wine_info, grape2idx, country2idx, winetype2idx

In [ ]:
wine_info, grape2idx, country2idx, winetype2idx = gen_labeled_data(wine_info)

In [52]:
wine_info.to_csv('/opt/ml/wine/data/wine_label_data.csv', index = False)

In [64]:
def check_price_in_review(text, price_vocab):
    price_vocab = set(price_vocab)
    text = text.lower()
    if any(word in text for word in price_vocab): return [1]
    else: return [0]

def marking_price_data(df, price_vocab):
    df.reset_index(inplace = True, drop = True)
    df['text'] = df['text'].fillna('')
    data = []
    for i in tqdm(range(len(df))):
        tmp = {}
        tmp['wine_id'] = df.loc[i,'wine_id']
        tmp['text'] = df.loc[i,'text']
        tmp['label'] = check_price_in_review(df.loc[i,'text'], price_vocab)
        data.append(tmp)
        
    return pd.DataFrame(data)


In [65]:
with open('/opt/ml/wine/data/price_vocab.json','r') as f: price_vocab = json.load(f)
price_data = marking_price_data(review_df, price_vocab)

100%|██████████| 7649958/7649958 [04:24<00:00, 28931.42it/s]


In [67]:
review_df
review_df['wine_id'] = review_df['wine_url'].map(item2idx)
review_df = review_df[review_df['wine_id'].isna()==False]
review_df['wine_id'] = review_df['wine_id'].astype('int').astype('category')

<ipython-input-67-9308ff83ffc9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df['wine_id'] = review_df['wine_id'].astype('int').astype('category')
<ipython-input-67-9308ff83ffc9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df.drop(['index','level_0','user_url','wine_url'], axis = 1, inplace = True)


In [70]:
def get_len_text(x):
    return len(x.split())

review_df = review_df.sort_values('wine_id')
review_df['length'] = review_df['text'].apply(get_len_text)

In [72]:
review_df = review_df.sort_values(['wine_id', 'length'])

In [98]:
def merge_short_review(df, threshold):
    df.reset_index(inplace = True, drop = True)
    data = []

    prv_text = ''
    prv_id = df.loc[0, 'wine_id']
    
    for i in tqdm(range(len(df))):
        tmp = {}
        
        wine_id = df.loc[i, 'wine_id']
        length = df.loc[i, 'length']
        text = df.loc[i, 'text']
        
        if length <= threshold: #######short text
            if (length + len(prv_text.split()) > threshold):
                if wine_id == prv_id:
                    text += prv_text
                    length += len(prv_text.split())

                tmp['wine_id'] = wine_id
                tmp['text'] = text
                tmp['length'] = length 
                data.append(tmp)
                prv_text = text
                prv_id = wine_id
                
            else: prv_text += text
        else: #######long text
            tmp['wine_id'] = wine_id
            tmp['text'] = text
            tmp['length'] = length
            data.append(tmp)
            prv_text = text
            prv_id = wine_id
        
    result = pd.DataFrame(data)
    result = result[result['length'] > 2]
    result.reset_index(inplace = True, drop = True)
    
    print(f"Before : {len(df)}")
    print(f"After : {len(result)}")
    return result

In [99]:
review_df = merge_short_review(review_df, 6)

  0%|          | 0/7147151 [00:00<?, ?it/s]

100%|██████████| 7147151/7147151 [08:06<00:00, 14698.91it/s] 


Before : 7147151
After : 6721655


In [91]:
labeled_data = pd.merge(review_df, wine_info, on = 'wine_id', how = 'inner')
labeled_text

,wine_id,text,length,country,grapes,winetype,grape_label,winetype_label,country_label
0,40,Amazing,1,Spain,[tempranillo],Red wine,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
1,40,Vinaso,1,Spain,[tempranillo],Red wine,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
2,40,Vinazo,1,Spain,[tempranillo],Red wine,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
3,40,👍,1,Spain,[tempranillo],Red wine,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,40,Powerful!,1,Spain,[tempranillo],Red wine,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...
1492925,74913,Our 1st bottle of the night and we decided to ...,93,France,"[cabernet_sauvignon, cabernet_franc, malbec, m...",Red wine,"[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1492926,74913,4.2 - very few things in life can beat the nos...,94,France,"[cabernet_sauvignon, cabernet_franc, malbec, m...",Red wine,"[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1492927,74913,A very good old wine of my vintage from an auc...,94,France,"[cabernet_sauvignon, cabernet_franc, malbec, m...",Red wine,"[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1492928,74913,"Very nice! One of the best, pleasant, fruity f...",94,France,"[cabernet_sauvignon, cabernet_franc, malbec, m...",Red wine,"[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
